<a href="https://colab.research.google.com/github/cvillarr123/trading/blob/master/notebook/ForexRoberto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install alpha_vantage finnhub-python pandas

In [ ]:
import pandas as pd 
import json
from pprint import pprint
from datetime import datetime
import dateutil.parser
from pytz import timezone
from dateutil import tz
from alpha_vantage.foreignexchange import ForeignExchange
import finnhub
# Setup client
alpha_client = ForeignExchange(key='RCMEEIVUZYDWGCDH', output_format='pandas')
finnhub_client = finnhub.Client(api_key="c2v8lvaad3i9mrpv2ru0")
pd.set_option('mode.chained_assignment', None)

In [ ]:
def convert_date_to_unix(p_date):
  gmt5 = timezone('America/Guayaquil')
  naive_ts = datetime.strptime(p_date, '%Y-%m-%d %H:%M:%S')
  local_ts = gmt5.localize(naive_ts)
  epoch_ts = local_ts.timestamp()
  final_date = str(epoch_ts).split(".")[0] 
  return int(final_date)

In [ ]:
def convert_unix_to_date(p_unix):
  tz = pytz.timezone('America/Guayaquil')
  dt = datetime.fromtimestamp(p_unix, tz)
  return dt.strftime('%Y-%m-%d %H:%M:%S')

In [ ]:
def format_candles_alpha(p_df):
  p_df.columns=['close', 'high', 'low', 'open', 'status', 'date']
  fi = p_df[['open', 'high', 'low', 'close','date']]
  fi.loc[:,'date'] = fi['date'].apply(convert_unix_to_date)
  fo = fi.set_index('date')
  return fo


In [ ]:
def fx_homologate(source, function, currency_from, currency_to, type_exchange, period, date_ini, date_end):
  data = ""
  if function == 'EXCHANGE':
    if source == 'ALPHA':
      res, _ = alpha_client.get_currency_exchange_rate(from_currency=currency_from,to_currency=currency_to)
      data = res
      data.columns=['From_Currency_Code','From_Currency_Name', 'To_Currency_Code', 'To_Currency_Name',
                    'Exchange_Rate', 'Last_Refreshed', 'Time_Zone', 'Bid_Price', 'Ask_Price']
    elif source == 'FINNHUB':
      res = finnhub_client.forex_rates(base=currency_from)
      exchange = res['quote']
      xe = pd.DataFrame([exchange])
      format_alpha = {'From_Currency_Code':currency_from,'From_Currency_Name':'',
              'To_Currency_Code':currency_to, 'To_Currency_Name':'',
             'Exchange_Rate':xe[currency_to], 'Last_Refreshed': datetime.utcnow(),
              'Time_Zone':'UTC', 'Bid_Price':'', 'Ask_Price':''}
      data = pd.DataFrame(format_alpha)
  elif function == 'CANDLES':
    if period == '1' or period == '5' or period == '15' or period == '30' or period == '60':
      if source == 'ALPHA':
        period = period+'min'
        data, _ = alpha_client.get_currency_exchange_intraday(from_symbol=currency_from, to_symbol=currency_to, interval=period)
        data.columns=['open', 'high', 'low', 'close']
      elif source == 'FINNHUB':
        d = finnhub_client.forex_symbols(type_exchange)
        df = pd.DataFrame(d)
        record = df[df['displaySymbol'].str.match(currency_from+'/'+currency_to)]
        res = finnhub_client.forex_candles(record.symbol, period, convert_date_to_unix(date_ini), convert_date_to_unix(date_end))
        if res["s"] == 'no_data':
          data = pd.DataFrame.from_dict(res, orient="index")
        else:
          data = format_candles_alpha(pd.DataFrame(res))
    elif period == 'D' or period == 'W' or period == 'M':
      if source == 'ALPHA':
        if period == 'D':
          data, _ = alpha_client.get_currency_exchange_daily(from_symbol=currency_from, to_symbol=currency_to)
          data.columns=['open', 'high', 'low', 'close']
        if period == 'W':
          data, _ = alpha_client.get_currency_exchange_weekly(from_symbol=currency_from, to_symbol=currency_to)
          data.columns=['open', 'high', 'low', 'close']
        if period == 'M':
          data, _ = alpha_client.get_currency_exchange_monthly(from_symbol=currency_from, to_symbol=currency_to)
          data.columns=['open', 'high', 'low', 'close']
      elif source == 'FINNHUB':
        d = finnhub_client.forex_symbols(type_exchange)
        df = pd.DataFrame(d)
        record = df[df['displaySymbol'].str.match(currency_from+'/'+currency_to)]
        res = finnhub_client.forex_candles(record.symbol, period, convert_date_to_unix(date_ini), convert_date_to_unix(date_end))
        if res["s"] == 'no_data':
          data = pd.DataFrame.from_dict(res, orient="index")
        else:
          data = format_candles_alpha(pd.DataFrame(res))
  return data

In [ ]:
#Lista de type_exchange:
#["oanda","fxcm","forex.com","fxpro","icmtrader","pepperstoneuk","ic markets","octafx","fxpig","pepperstone"]

# Al dia 14 Julio 2021 esta integrada en formato homologado de ALPHA para los metodos EXCHANGE y CANDLES para el proveedor FOREX
# FINNHUB no entrega volumetria para FOREX, pero si para los demas proveedores
fe = fx_homologate('FINNHUB', 'CANDLES', 'GBP', 'AUD', 'forex.com', 'D', '2021-07-10 00:00:00', '2021-07-12 00:00:00')
#fe = fx_homologate('ALPHA', 'EXCHANGE', 'GBP', 'AUD', 'forex.com', '1', '2021-07-10 00:00:00', '2021-07-12 00:00:00')
fe

,open,high,low,close
date,,,,
2021-07-11 19:00:00,1.85776,1.86104,1.85302,1.85593
2021-07-12 19:00:00,1.85676,1.85920,1.85061,1.85510
2021-07-13 19:00:00,1.85491,1.86203,1.85079,1.85270
2021-07-14 19:00:00,1.85216,1.85318,1.85152,1.85263


In [ ]:
print(convert_unix_to_date(1626134400))

2021-07-12 19:00:00


In [ ]:
convert_date_to_unix("2021-07-12 19:00:00")


1626134400